In [1]:
import ee
import folium

In [2]:
# opens browser to request authenticatication token (OAUTH2) and store it.
ee.Authenticate()

Enter verification code: 4/1AY0e-g7g5klvr1fOjhux3LX_BFhYkRGyLotmd97Ca5ei3fUyMfSEt5MSOf0

Successfully saved authorization token.


In [3]:
# uses your stored auth token to login to ee.
ee.Initialize()

In [4]:
class EngineMap(folium.Map):
    """
    Subclass of folium Map for adding EE images. 
    Here self is a folium.Map class instance.
    Adding additional functions to this class type.
    """
    
    def __init__(self, location=None, zoom_start=None, **kwargs):
        # Inherit from parent class
        super().__init__(location=location, zoom_start=zoom_start, **kwargs)
    
    
    def add_ee_vector(self, feature, **kwargs):
        """
        Add a google earth engine vector feature to a folium map.
        """
        # Create vector feature
        feature = folium.GeoJson(
            data=feature.geometry().getInfo(),
            **kwargs,
        )
        
        # Add vector to Map and return self to allow chaining
        self.add_child(feature)
        return self
    
    
    def add_ee_raster(self, image, vis_params={}, **kwargs):
        """
        Add a google earth engine raster layer to a folium map.
        """       
        # Handle ImageCollections and Images
        if isinstance(image, ee.ImageCollection):
            ee_image = image.mosaic()

        # Get the JSON instructions to show image tiles
        map_id_dict = image.getMapId(vis_params)
        
        # Get url of the raster tiles
        tiles = map_id_dict['tile_fetcher'].url_format
        
        # Create a folium raster layer and add to self
        raster = folium.raster_layers.TileLayer(
            tiles=tiles,
            attr=kwargs.get("attr", "TODO-attr"),
            name=kwargs.get("name", "test"),
            overlay=True,
            control=True,
        )
        
        # Add raster to Map and return self to allow chaining
        self.add_child(raster)
        return self

### Default visualization of global forest change

In [44]:
# Load Hansen forest dataset raster
gfc2014 = ee.Image('UMD/hansen/global_forest_change_2015')

# Create map on Taiwan
forest_map = EngineMap(location=[23.69, 120.96], zoom_start=7)

# Add raster to map
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan"
)

# Display map
forest_map

### Show tree cover

In [45]:
# Create map displaying forest cover in the year 2000 as a grayscale image
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['treecover2000']
    }
)

# Display map
forest_map

### Tree cover (Landsat bands)

In [46]:
# Add raster to map using Landsat bands 5, 4, and 3
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['last_b50', 'last_b40', 'last_b30'],
    }
)

# Display map
forest_map

### Forest cover - loss and gain

In [47]:
# Add raster to map
# make loss the first band (red), treecover2000 the second band (green), and gain the third band (blue)
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['loss', 'treecover2000', 'gain'],
    }
)

# Display map
forest_map

### Fix image visualization parameter

In [48]:
# Add raster to map, specify visualization parater
# forest loss will show up as bright red and forest gain will show up as bright blue
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['loss', 'treecover2000', 'gain'], 
        "max": [1, 255, 1]
    }
)

# Display map
forest_map

### Add palette

In [49]:
# Use a green palette to display the forest extent image
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['treecover2000'], 
        "palette": ['000000','00FF00'], 
    }
)

# Display map
forest_map

In [50]:
# Use max=100 gives it a brighter green color
forest_map.add_ee_raster(
    image = gfc2014,
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['treecover2000'], 
        "palette": ['000000','00FF00'], 
        "max": 100
    }
)

# Display map
forest_map

### Masking

In [51]:
# Create a new map for visualization (with masking)
forest_map2 = EngineMap(location=[23.69, 120.96], zoom_start=7)

# Add raster
forest_map2.add_ee_raster(
    image = gfc2014.mask(gfc2014),
    attr = "Catherine Lan",
    vis_params = {
        "bands": ['treecover2000'], 
        "palette": ['000000','00FF00'],
        "max": 100
    }
)

# Display map
forest_map2

### Finalize map 

In [52]:
# Create final map
# Using select() to put bands, forest cover, loss and gain into their own images
final_forest_map = EngineMap(location=[23.69, 120.96], zoom_start=7)

# Select bands to create 3 images
treecover = gfc2014.select('treecover2000')
loss = gfc2014.select('loss')
gain = gfc2014.select('gain')

# Add raster for each image

# tree cover
final_forest_map.add_ee_raster(
    image = treecover.updateMask(treecover),
    name = 'Forest Cover Taiwan 2000',
    attr = "Catherine Lan",
    vis_params = {
        "palette": ['000000', '00FF00'],
        "max": 100
    }
)

# loss
final_forest_map.add_ee_raster(
    image = loss.updateMask(loss),
    name = 'Forest Loss Taiwan 2000',
    attr = "Catherine Lan",
    vis_params = {
        "palette": ['FF0000'],
    }
)

# gain
final_forest_map.add_ee_raster(
    image = gain.updateMask(gain), #masking
    name = 'Forest Gain Taiwan 2000', #name
    attr = "Catherine Lan", #attribution
    vis_params = {
        "palette": ['0000FF'] #bright blue
    }
)

# Display map
final_forest_map